In [1]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
import psycopg2

%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5432',
                host = 'localhost',                
                database = 'inicudb')

cur  = con.cursor()

In [38]:


uhidPatient = "('200700003','200700160','200700148','200700206','200700207')" 
caseType = 'Death'
conditionCase = "(dischargestatus = 'Death' or dischargestatus = 'LAMA' )"
#uhidPatient = "RSHI.0000012984"
#caseType = 'Discharge'
#conditionCase = "(dischargestatus = 'Discharge')"
seperator = '_'

In [3]:
#base = pd.read_csv("/Users/harpreet/Videos/data_22_june.csv",  nrows=999999)

In [4]:
#ds = base[~base.starttime.notna()]

In [5]:
#ds.hour_series.head()

In [6]:
#base[~base.hour_series.notna()]

In [7]:
#death_base = base[base.dischargestatus == 1]

In [8]:
#discharge_base = base[base.dischargestatus == 0]

In [9]:
#death_base.uhid.unique()


In [10]:
#discharge_base.uhid.unique()

In [11]:
#dea = death_base.groupby('uhid').head(1)
#dis = discharge_base.groupby('uhid').head(1)

In [12]:
#checkingUHIDData = death_base[death_base.uhid==uhidPatient]

In [13]:
#checkingUHIDData.to_csv(uhidPatient+".csv")

In [14]:
#print(dea.gestationweekbylmp, dea.birthweight,dea.uhid)

In [15]:
#print(dea.gestationweekbylmp, dea.birthweight,dea.uhid)

In [16]:
#lesstha26 = dis[(dis.gestationweekbylmp <= 26) & (dis.birthweight<=1500)]
#between26and28 = dis[(dis.gestationweekbylmp > 26) & (dis.gestationweekbylmp <= 28) & (dis.birthweight<=1500)]
#between28and32 = dis[(dis.gestationweekbylmp > 28) & (dis.gestationweekbylmp <= 32) & (dis.birthweight<=1500)]
#greaterthan32 = dis[(dis.gestationweekbylmp > 32) & (dis.birthweight<=3000) & (dis.birthweight>=2500)]

In [17]:
#(lesstha26.birthweight)




In [18]:
#(between26and28.birthweight)

In [19]:
#(between28and32.birthweight)

In [20]:
#len(greaterthan32)

In [39]:
cur10 = con.cursor()
cur10.execute("SELECT t1.uhid,t1.dateofbirth,t1.timeofbirth,t1.timeofadmission,t1.dateofadmission,t1.gender,t1.birthweight,t1.birthlength,t1.birthheadcircumference,t1.inout_patient_status,t1.gestationweekbylmp,t1.gestationdaysbylmp,t1.baby_type,t1.central_temp,t1.dischargeddate,t2.apgar_onemin,t2.apgar_fivemin,t2.apgar_tenmin,t3.motherage,t4.conception_type,t4.mode_of_delivery,t5.steroidname,t5.numberofdose,t1.dischargestatus  FROM kalawati.baby_detail AS t1 LEFT JOIN kalawati.birth_to_nicu AS t2 ON t1.uhid=t2.uhid LEFT JOIN kalawati.parent_detail AS t3 ON t1.uhid=t3.uhid LEFT JOIN kalawati.antenatal_history_detail AS t4 ON t1.uhid=t4.uhid LEFT JOIN kalawati.antenatal_steroid_detail AS t5 ON t1.uhid=t5.uhid where timeofadmission is not null and timeofbirth is not null and t1.uhid IN "+uhidPatient+";")
cols10 = list(map(lambda x: x[0], cur10.description))
fixed = pd.DataFrame(cur10.fetchall(),columns=cols10)

In [40]:
fixed

,uhid,dateofbirth,timeofbirth,timeofadmission,dateofadmission,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,dischargeddate,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus
0,200700148,2020-07-03,"01,40,PM","03,29,PM",2020-07-03,Female,1130.0,36.0,27.0,Out Born,...,2020-07-09 23:00:00,8.0,8,9,None,None,NVD,None,NaN,None
1,200700003,2020-07-01,"04,07,AM","04,28,AM",2020-07-01,Male,1390.0,39.0,30.0,In Born,...,2020-07-09 23:00:00,8.0,8,9,None,natural,NVD,Dexa,1.0,None
2,200700160,2020-07-06,"01,50,PM","02,40,PM",2020-07-06,Male,800.0,40.0,26.0,In Born,...,2020-07-09 23:00:00,NaN,8,9,None,None,NVD,None,NaN,None
3,200700206,2020-07-04,"06,33,PM","07,08,PM",2020-07-04,Female,1700.0,46.0,31.0,In Born,...,2020-07-09 23:00:00,5.0,7,9,None,ivf,LSCS,None,NaN,None
4,200700207,2020-07-04,"06,35,PM","07,09,PM",2020-07-04,Female,1700.0,44.0,31.0,In Born,...,2020-07-09 23:00:00,5.0,8,9,None,ivf,LSCS,Dexa,NaN,None


In [7]:
len(fixed)

5

In [41]:
s1 = set(fixed.uhid.unique())

In [42]:
cur11 = con.cursor()
cur11.execute("SELECT DISTINCT(uhid) FROM kalawati.baby_detail WHERE dateofadmission >= '2018-07-01' AND dateofadmission <= '2020-08-25' and UHID IN  (select distinct(uhid) from kalawati.babyfeed_detail where uhid in  ( select distinct(uhid) from kalawati.baby_visit where uhid in (select  distinct(uhid) from kalawati.nursing_vitalparameters where uhid in ( select distinct(uhid) from kalawati.device_monitor_detail UNION select distinct(uhid) from kalawati.device_monitor_detail_dump)))) and  isreadmitted is not true and gestationweekbylmp is not null and birthweight is not null and uhid IN "+uhidPatient+";")
cols11 = list(map(lambda x: x[0], cur11.description))
uhids = pd.DataFrame(cur11.fetchall(),columns=cols11)

In [43]:
print(len(uhids.uhid.unique()))

5


In [44]:
uhids

,uhid
0,200700003
1,200700148
2,200700160
3,200700206
4,200700207


In [45]:
s2 = set(uhids.uhid.unique())

In [46]:
l = list(s1 - s1.intersection(s2))

In [47]:
len(uhids)

5

In [48]:
fixed.set_index('uhid',inplace=True)

In [49]:
df = fixed.drop(l)

In [50]:
len(df.drop_duplicates())

5

In [51]:
df.reset_index(inplace=True)

In [20]:
len(df.uhid.unique())

5

In [34]:
#dates_detail = ds.copy()

In [52]:
dates_detail = df.copy()

In [53]:
dates_detail.head()

,uhid,dateofbirth,timeofbirth,timeofadmission,dateofadmission,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,dischargeddate,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus
0,200700148,2020-07-03,"01,40,PM","03,29,PM",2020-07-03,Female,1130.0,36.0,27.0,Out Born,...,2020-07-09 23:00:00,8.0,8,9,None,None,NVD,None,NaN,None
1,200700003,2020-07-01,"04,07,AM","04,28,AM",2020-07-01,Male,1390.0,39.0,30.0,In Born,...,2020-07-09 23:00:00,8.0,8,9,None,natural,NVD,Dexa,1.0,None
2,200700160,2020-07-06,"01,50,PM","02,40,PM",2020-07-06,Male,800.0,40.0,26.0,In Born,...,2020-07-09 23:00:00,NaN,8,9,None,None,NVD,None,NaN,None
3,200700206,2020-07-04,"06,33,PM","07,08,PM",2020-07-04,Female,1700.0,46.0,31.0,In Born,...,2020-07-09 23:00:00,5.0,7,9,None,ivf,LSCS,None,NaN,None
4,200700207,2020-07-04,"06,35,PM","07,09,PM",2020-07-04,Female,1700.0,44.0,31.0,In Born,...,2020-07-09 23:00:00,5.0,8,9,None,ivf,LSCS,Dexa,NaN,None


In [54]:
def second_addition(x):
    try:
        if x.split(",")[2] == 'PM':
            if x.split(",")[0] != '12':
                return (12 + float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
            else:
                return (float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
        else:
            return (float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
    except:
        
        if x.split(" ")[1] == 'PM':
            if (x.split(" ")[0]).split(":")[0] != '12':
                return (12 + float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
            else:
                return (float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
        else:
            return (float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
    
        

In [55]:
def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

def split_hour(x):
    try:
        t=str(x)
        return t.split(":")[0]
    except:
        pass
    
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass
    

In [56]:
dates_detail['add_seconds'] = dates_detail['timeofbirth'].apply(second_addition)
dates_detail['add_seconds_admission'] = dates_detail['timeofadmission'].apply(second_addition)

In [57]:
from datetime import timedelta

In [58]:
def actual_birthdate(x,y):
    try:
        return pd.to_datetime(x) + timedelta(seconds=y)
    except:
        pass

In [59]:
dates_detail['actual_DOB'] = dates_detail.apply(lambda x: actual_birthdate(x['dateofbirth'], x['add_seconds']), axis=1)

In [60]:
dates_detail['actual_DOA'] = dates_detail.apply(lambda x: actual_birthdate(x['dateofadmission'], x['add_seconds_admission']), axis=1)

In [61]:
dd = dates_detail[['actual_DOB','uhid','dischargeddate','actual_DOA','gender','birthweight','birthlength','birthheadcircumference','inout_patient_status'
,'gestationweekbylmp','gestationdaysbylmp','baby_type','central_temp','apgar_onemin',
'apgar_fivemin','apgar_tenmin','motherage','conception_type','mode_of_delivery','steroidname',
'numberofdose','dischargestatus','birthlength','birthheadcircumference','inout_patient_status'
,'gestationweekbylmp','gestationdaysbylmp','baby_type','central_temp','apgar_onemin',
'apgar_fivemin','apgar_tenmin','motherage','conception_type','mode_of_delivery','steroidname',
'numberofdose']]

In [62]:
dd.dropna(subset=['dischargeddate'],inplace=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
dd['los'] = dates_detail['dischargeddate'] - dates_detail['actual_DOA']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
def day(x):
    return x.total_seconds()/86400

In [65]:
dd.los

0   6 days 07:31:00
1   8 days 18:32:00
2   3 days 08:20:00
3   5 days 03:52:00
4   5 days 03:51:00
Name: los, dtype: timedelta64[ns]

In [66]:
dd['day_1'] = dd['los'].apply(day)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [67]:
dd = dd[dd['day_1']>=0]

In [68]:
dd.sort_values(by = ['actual_DOA'],inplace=True)

In [69]:
dd.drop_duplicates(subset=['uhid'],keep='first',inplace=True)

In [70]:
dd

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
1,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222
0,2020-07-03 13:40:00,200700148,2020-07-09 23:00:00,2020-07-03 15:29:00,Female,1130.0,36.0,27.0,Out Born,28,...,8.0,8,9,None,None,NVD,None,NaN,6 days 07:31:00,6.313194
3,2020-07-04 18:33:00,200700206,2020-07-09 23:00:00,2020-07-04 19:08:00,Female,1700.0,46.0,31.0,In Born,31,...,5.0,7,9,None,ivf,LSCS,None,NaN,5 days 03:52:00,5.161111
4,2020-07-04 18:35:00,200700207,2020-07-09 23:00:00,2020-07-04 19:09:00,Female,1700.0,44.0,31.0,In Born,31,...,5.0,8,9,None,ivf,LSCS,Dexa,NaN,5 days 03:51:00,5.160417
2,2020-07-06 13:50:00,200700160,2020-07-09 23:00:00,2020-07-06 14:40:00,Male,800.0,40.0,26.0,In Born,34,...,NaN,8,9,None,None,NVD,None,NaN,3 days 08:20:00,3.347222


In [71]:
dt = pd.DataFrame()

In [72]:
dt

""


In [73]:
dd

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
1,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222
0,2020-07-03 13:40:00,200700148,2020-07-09 23:00:00,2020-07-03 15:29:00,Female,1130.0,36.0,27.0,Out Born,28,...,8.0,8,9,None,None,NVD,None,NaN,6 days 07:31:00,6.313194
3,2020-07-04 18:33:00,200700206,2020-07-09 23:00:00,2020-07-04 19:08:00,Female,1700.0,46.0,31.0,In Born,31,...,5.0,7,9,None,ivf,LSCS,None,NaN,5 days 03:52:00,5.161111
4,2020-07-04 18:35:00,200700207,2020-07-09 23:00:00,2020-07-04 19:09:00,Female,1700.0,44.0,31.0,In Born,31,...,5.0,8,9,None,ivf,LSCS,Dexa,NaN,5 days 03:51:00,5.160417
2,2020-07-06 13:50:00,200700160,2020-07-09 23:00:00,2020-07-06 14:40:00,Male,800.0,40.0,26.0,In Born,34,...,NaN,8,9,None,None,NVD,None,NaN,3 days 08:20:00,3.347222


In [74]:
dd.columns

Index(['actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA', 'gender',
       'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength',
       'birthheadcircumference', 'inout_patient_status', 'gestationweekbylmp',
       'gestationdaysbylmp', 'baby_type', 'central_temp', 'apgar_onemin',
       'apgar_fivemin', 'apgar_tenmin', 'motherage', 'conception_type',
       'mode_of_delivery', 'steroidname', 'numberofdose', 'los', 'day_1'],
      dtype='object')

In [75]:
dd.dischargeddate

1   2020-07-09 23:00:00
0   2020-07-09 23:00:00
3   2020-07-09 23:00:00
4   2020-07-09 23:00:00
2   2020-07-09 23:00:00
Name: dischargeddate, dtype: datetime64[ns]

In [58]:
dd.dischargeddate.iloc[0]

Timestamp('2018-08-01 20:40:04.099000')

In [59]:
dd.actual_DOA.iloc[0]

Timestamp('2018-07-10 12:30:00')

In [60]:
x = dd[dd['uhid']==uhidPatient]

In [61]:
x

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
0,2018-07-10 08:57:00,RSHI.0000012088,2018-08-01 20:40:04.099,2018-07-10 12:30:00,Female,1408.0,None,None,In Born,28,...,7,8,None,31,ivf,LSCS,None,None,22 days 08:10:04.099000,22.340325


In [62]:
(x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()

1930204.099

In [76]:
import math
for i in dd.uhid.unique():
    x = dd[dd['uhid']==i]
    time_s = []
    n = math.ceil((x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()/60)
    try:
        x = pd.concat([x]*int(n))

        for i in range(0,len(x)):
            #print(i)
            time_s.append(x['actual_DOA'].iloc[i] + timedelta(seconds = i*60))
            #t['hour_series'].loc[i] = t['actual_DOA'].iloc[i] + timedelta(hours = i)

        x['hour_series'] = time_s
    except Exception as e:
        print(i,n, e)
        continue
    dt = dt.append(x,ignore_index=True)


In [79]:
len(dt)

41406

In [ ]:
dt

In [ ]:
dt

In [78]:
dt.drop_duplicates(subset=['uhid','hour_series'],inplace=True)

In [ ]:
dt

In [80]:
cur1 = con.cursor()
cur1.execute("select distinct(b.uhid),l.conception_type, b.gender, b.dateofadmission, b.dischargestatus,b.birthweight,b.weight_galevel,b.weight_centile,b.birthlength,b.birthheadcircumference, b.inout_patient_status, b.gestationweekbylmp, b.gestationdaysbylmp,round( CAST((b.gestationweekbylmp + b.gestationdaysbylmp/7::float) as numeric),2) as Gestation,b.dischargeddate, b.admissionweight, b.baby_type,b.baby_number, b.branchname ,DATE_PART('day',b.dischargeddate - b.dateofadmission) as LOS,c.apgar_onemin, c.apgar_fivemin,c.apgar_tenmin, c.resuscitation,d.isantenatalsteroidgiven, d.mode_of_delivery,z.motherage,e.jaundicestatus as JAUNDICE,  f.eventstatus as SEPSIS,f.progressnotes, g.eventstatus as RDS, g.progressnotes,y.eventstatus as ASPHYXIA,y.progressnotes from kalawati.baby_detail as b left join kalawati.birth_to_nicu as c on b.uhid = c.uhid and b.episodeid = c.episodeid left join kalawati.parent_detail as z on b.uhid = z.uhid and b.episodeid = z.episodeid left join kalawati.antenatal_history_detail as d on b.uhid = d.uhid and b.episodeid = d.episodeid left join kalawati.sa_jaundice AS e ON b.uhid = e.uhid and e.jaundicestatus = 'Yes' and e.phototherapyvalue='Start' left join kalawati.sa_infection_sepsis AS f ON b.uhid = f.uhid and f.eventstatus = 'yes' and f.episode_number = 1 left join kalawati.sa_cns_asphyxia AS y ON b.uhid = y.uhid and y.eventstatus = 'yes' and y.episode_number = 1 left join kalawati.antenatal_history_detail as l on b.uhid=l.uhid left join kalawati.sa_resp_rds AS g ON b.uhid = g.uhid and g.eventstatus = 'Yes' and g.episode_number = 1 and g.uhid IN (select distinct(h.uhid) from kalawati.respsupport AS h where h.eventname='Respiratory Distress' and (h.rs_vent_type ='Mechanical Ventilation' OR h.rs_vent_type ='HFO') UNION select distinct(i.uhid) from kalawati.sa_resp_rds AS i where i.sufactantname is not null) and b.uhid IN "+uhidPatient+" order by b.dateofadmission")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
#ds.sort_values('modificationtime',inplace=True)

In [81]:
len(dt.uhid.unique())

5

In [82]:
data = ds.copy()

In [83]:
len(ds)

3737

In [85]:
dt

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series
0,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:28:00
1,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:29:00
2,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:30:00
3,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:31:00
4,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:32:00
5,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:33:00
6,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:34:00
7,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:35:00
8,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:36:00
9,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,...,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:37:00


In [86]:
ids = dt.uhid.unique()

In [87]:
len(dt)

41406

In [88]:
data.drop_duplicates('uhid',keep='first',inplace=True)

In [89]:
df = pd.DataFrame(columns=data.columns)

In [90]:
for i in ids:
    x = data[data['uhid']==i]
    df = df.append(x,ignore_index=True)

In [ ]:
#d = pd.read_csv('baseline.csv')

In [ ]:
#len(d)

In [ ]:
#s = set(d['uhid'].unique())

In [91]:
pd.set_option('display.max_columns',100)

In [ ]:
#s1 = set(ids)

In [92]:
ds = pd.DataFrame()

In [93]:
ds = dt.copy()

In [ ]:
#ds.to_csv('test_baseline.csv')

In [94]:
uhid = ds.uhid.unique()

In [95]:
dt = pd.DataFrame()

In [96]:
for i in uhid:
    x = dd[dd['uhid']==i]
    time_s = []
    n = math.ceil((x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()/60)
    try:
        x = pd.concat([x]*int(n))
        for i in range(0,len(x)):
            #print(i)
            time_s.append(x['actual_DOA'].iloc[i] + timedelta(seconds = i*60))
            #t['hour_series'].loc[i] = t['actual_DOA'].iloc[i] + timedelta(hours = i)

        x['hour_series'] = time_s
    except Exception as inst:
        print(inst)
        continue
    dt = dt.append(x,ignore_index=True)


In [97]:
dd['uhid']

1    200700003
0    200700148
3    200700206
4    200700207
2    200700160
Name: uhid, dtype: object

In [ ]:
len(dt)

In [ ]:
x

In [ ]:
uhid

In [98]:
dt['hour_series'].apply(split_hour)

0        2020-07-01 04
1        2020-07-01 04
2        2020-07-01 04
3        2020-07-01 04
4        2020-07-01 04
5        2020-07-01 04
6        2020-07-01 04
7        2020-07-01 04
8        2020-07-01 04
9        2020-07-01 04
10       2020-07-01 04
11       2020-07-01 04
12       2020-07-01 04
13       2020-07-01 04
14       2020-07-01 04
15       2020-07-01 04
16       2020-07-01 04
17       2020-07-01 04
18       2020-07-01 04
19       2020-07-01 04
20       2020-07-01 04
21       2020-07-01 04
22       2020-07-01 04
23       2020-07-01 04
24       2020-07-01 04
25       2020-07-01 04
26       2020-07-01 04
27       2020-07-01 04
28       2020-07-01 04
29       2020-07-01 04
             ...      
41376    2020-07-09 22
41377    2020-07-09 22
41378    2020-07-09 22
41379    2020-07-09 22
41380    2020-07-09 22
41381    2020-07-09 22
41382    2020-07-09 22
41383    2020-07-09 22
41384    2020-07-09 22
41385    2020-07-09 22
41386    2020-07-09 22
41387    2020-07-09 22
41388    20

In [99]:
dt['hour'] = dt['hour_series'].apply(split_hour)

In [100]:
dt['day'] = dt['hour_series'].apply(split_date_1)

In [101]:
len(dt.uhid.unique())

5

In [102]:
#output
cur9= con.cursor()
cur9.execute("SELECT t1.uhid,t1.abdomen_girth,t1.urine,t1.stool,t1.stool_passed, t1.entry_timestamp FROM kalawati.nursing_intake_output AS t1 where t1.uhid IN "+uhidPatient+";")
cols9 = list(map(lambda x: x[0], cur9.description))
output = pd.DataFrame(cur9.fetchall(),columns=cols9)

In [103]:
output

,uhid,abdomen_girth,urine,stool,stool_passed,entry_timestamp
0,200700003,25,None,None,None,2020-07-01 12:04:28.197000+05:30
1,200700003,25,20,None,None,2020-07-02 07:53:31.759000+05:30
2,200700003,25,None,None,None,2020-07-02 10:18:36.272000+05:30
3,200700003,25,20,Medium,True,2020-07-02 16:03:02.866000+05:30
4,200700003,25,None,None,None,2020-07-01 05:55:05.361000+05:30
5,200700003,25,None,None,None,2020-07-01 09:16:58.633000+05:30
6,200700003,25,None,None,None,2020-07-01 10:38:11.045000+05:30
7,200700003,25,None,None,None,2020-07-01 14:15:56.729000+05:30
8,200700003,25,20,Medium,True,2020-07-02 14:26:03.612000+05:30
9,200700003,25,20,None,None,2020-07-01 17:50:06.231000+05:30


In [ ]:
dt

In [104]:
output_d = pd.DataFrame()

In [105]:
for i in uhid:
    x = output[output['uhid']==i]
    output_d = output_d.append(x,ignore_index=True)

In [106]:
output_d.entry_timestamp = output_d.entry_timestamp + timedelta(seconds=19800)

In [ ]:
output_d

In [107]:
output_d['hour'] = output_d.entry_timestamp.apply(split_hour)
output_d['day'] = output_d.entry_timestamp.apply(split_date_1)

In [ ]:
output_d

In [108]:
output_d.sort_values(by=['uhid','entry_timestamp'],inplace=True)

In [ ]:
len(output_d)

In [109]:
test = output_d.drop_duplicates(subset=['uhid','hour'],keep='last')

In [ ]:
test.head()

In [ ]:
dt.head()

In [110]:
dh = pd.merge(dt,test,on=['uhid','hour'],how='left')

In [ ]:
dh

In [ ]:
#dh.sort_values(by=['uhid','entry_timestamp'],inplace=True)

In [111]:
dh.sort_values('hour_series')

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y
0,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:28:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
1,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:29:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
2,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:30:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
3,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:31:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
4,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:32:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
5,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:33:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
6,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:34:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
7,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:35:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
8,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:36:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
9,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:37:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN


In [112]:
dg = pd.DataFrame()

In [113]:
dh

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y
0,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:28:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
1,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:29:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
2,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:30:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
3,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:31:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
4,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:32:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
5,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:33:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
6,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:34:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
7,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:35:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
8,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:36:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN
9,2020-07-01 04:07:00,200700003,2020-07-09 23:00:00,2020-07-01 04:28:00,Male,1390.0,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,None,39.0,30.0,In Born,34,5,Twins,NaN,8.0,8,9,None,natural,NVD,Dexa,1.0,8 days 18:32:00,8.772222,2020-07-01 04:37:00,2020-07-01 04,2020-07-01,NaN,NaN,NaN,NaN,NaT,NaN


In [ ]:
len(dh)

In [ ]:
x.columns

In [ ]:
x.urine

In [114]:
for i in uhid:
    x = dh[dh['uhid']==i]
    #x.dropna(subset=['urine'],inplace=True)
    x['time_divide'] = 0
    print(len(x))
    startTime = None
    for j in range(len(x)):
        try:
            if j == 0:
                x.loc[j,'time_divide'] = 0.0                
                startTime = x.actual_DOA.iloc[j]
            else:
                if(x.urine.iloc[j] != None):
                    urine_value = float(x.urine.iloc[j])
                    if(~math.isnan(urine_value) and (urine_value >0)):
                        x.loc[j,'time_divide'] = (pd.to_datetime(str(x.entry_timestamp.iloc[j]).split("+")[0]) - startTime).total_seconds()
                        startTime = pd.to_datetime(str(x.entry_timestamp.iloc[j]).split("+")[0])
            #print(x.loc[j,'time_divide'])
        except Exception as e:
            print(j,'Exception is',e)
            continue
    dg = dg.append(x,ignore_index=True)    

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


12632
9091


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


7432
2512 Exception is could not convert string to float: 
2513 Exception is could not convert string to float: 
2514 Exception is could not convert string to float: 
2515 Exception is could not convert string to float: 
2516 Exception is could not convert string to float: 
2517 Exception is could not convert string to float: 
2518 Exception is could not convert string to float: 
2519 Exception is could not convert string to float: 
2520 Exception is could not convert string to float: 
2521 Exception is could not convert string to float: 
2522 Exception is could not convert string to float: 
2523 Exception is could not convert string to float: 
2524 Exception is could not convert string to float: 
2525 Exception is could not convert string to float: 
2526 Exception is could not convert string to float: 
2527 Exception is could not convert string to float: 
2528 Exception is could not convert string to float: 
2529 Exception is could not convert string to float: 
2530 Exception is could

In [115]:
dg.time_divide.unique()

array([    0.   , 67926.231,  9161.544, 15695.336, 14177.385,  6158.858,
        5412.405, 17213.568,  6338.285,  5819.254,  7532.576, 21921.13 ,
        7262.306, 10701.935,  4991.719,  5019.998,  8797.234,  8147.022,
        4016.963, 19465.942,  2975.048,  8440.239, 13833.14 , 14833.706,
        6189.245,  9454.75 ,  7432.724,  6293.251, 15225.602, 13523.818,
        8113.174, 15612.621,  3659.78 , 31335.109,  5371.569,  7324.922,
       15107.769,  7507.39 , 20784.043,  7091.707,  9748.085,  7214.051,
        5138.189, 13349.124, 14592.792, 17490.681,  4777.271,  8791.391,
       12067.345,  9325.955, 12267.071, 13249.638,  7078.558,  8521.531,
       14758.633,  7044.721, 13417.279,  8694.653,  5025.666,  7759.91 ,
       10783.802,  6088.963,  5909.657,  7627.733,  7482.896, 57868.352,
       15691.813, 14334.069,  7929.152, 13956.866, 21476.22 ,  8133.036,
       19077.778,  6187.489, 15640.928, 15090.716,  6883.871,  6174.077,
       10189.675,  6642.52 , 20170.397, 24312.161, 

In [ ]:
dg

In [116]:
dg = dg.sort_values('hour_series')

In [ ]:
dg

In [ ]:
dg.time_divide.unique()

In [ ]:
dg = dg.sort_values('hour_series')

In [ ]:
dg

In [ ]:
dg.columns


In [117]:
dg.to_csv('intermediate_checkpoint_apollo_5_9july.csv')

In [ ]:
dg